In [1]:
import mujoco
from mujoco import mjx
import jax
import mediapy as media



# Make model, data, and renderer
mj_model = mujoco.MjModel.from_xml_path('aloha/scene.xml')
mj_data = mujoco.MjData(mj_model)
renderer = mujoco.Renderer(mj_model)



mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.put_data(mj_model, mj_data)

# print(mj_data.qpos, type(mj_data.qpos))
# print(mjx_data.qpos, type(mjx_data.qpos), mjx_data.qpos.devices())


# enable joint visualization option:
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

duration = 3.8  # (seconds)
framerate = 30  # (Hz)

frames = []
mujoco.mj_resetData(mj_model, mj_data)
# while mj_data.time < duration:
#   mujoco.mj_step(mj_model, mj_data)
#   if len(frames) < mj_data.time * framerate:
#     renderer.update_scene(mj_data, scene_option=scene_option)
#     pixels = renderer.render()
#     frames.append(pixels)

# # Simulate and display video.
# media.show_video(frames, fps=framerate)

In [2]:


rng = jax.random.PRNGKey(0)
rng = jax.random.split(rng, 24)


fun=lambda rng: mjx_data.replace(qpos=jax.random.uniform(rng, (15,)))
fun_vmapped = jax.vmap(fun)
batch=fun_vmapped(rng)


jit_step = jax.jit(jax.vmap(mjx.step, in_axes=(None, 0)))
batch = jit_step(mjx_model, batch)


print(batch.qpos)